# 倒立振子のシミュレーションを Julia で行いたい

## はじめに

### 倒立振子とは

台車の上に棒が立っていて，その棒が倒れないようにしつつ台車を動かすという制御問題のことを倒立振子（とうりつしんし，inverted pendulum）といいます．

制御や強化学習の界隈の Hello World としてよく親しまれている問題です．倒立振子のシミュレーションを自分で組むと勉強になるとされています．

### Julia とは

主に科学計算の世界で使われている比較的新しい言語です．Python や Matlab を使ってもいいのですが，今回は Julia を使います．

## 倒立振子の運動方程式

シミュレーションをするには，運動方程式がわかっている必要があります．

運動方程式の導出は物理学の問題であって，制御の問題ではないのでここでは詳しくは述べません．[Excelで学ぶ振動基礎/運動方程式の立て方](https://edu.katzlab.jp/lec/vib7h/files/vib7h_B.pdf)に解析力学を使って導出する方法が解説されているので，そちらを参照してください．

ここでは以下のようなパラメータを考えます．

* 定数
  * 台車の質量 $M$ [kg]
  * 振子の質量 $m$ [kg]
  * 重力加速度 $g$ [m/s^2]
  * 台車から振子の重心までの距離 $l$ [m]

* 変数
  * 台車の位置 $x$ [m]
  * 振子の傾き $θ$ [rad]
  * 外力 $f$ [N] (制御入力 $u$ に相当する)

摩擦や空気抵抗は考えないものとしています．

このとき，結論だけ書いてしまうと，運動方程式は次の通りです．
$$
(M + m) \ddot{x} + m l \ddot{θ} \cos θ - m l \dot{θ}^2 \sin θ = f \\\\
(ml \cos θ) \ddot{x} + ml^2 \ddot{θ} + mgl \sin θ = 0
$$

## 線形化する

非線形なままだと扱いづらくて困るので，似ている線形システムに置き換えてしまいます．ここでは $θ$ が小さいと仮定して線形化します．これは，振子の傾きがそんなに大きくないということを意味します．

* 三角関数はゼロの周りの接線により近似をします．このとき，$\sin θ ≈ θ, \cos θ ≈ 1$ と近似できます．
* また，$θ$ が小さいと仮定したので $\dot{θ}^2 ≈ 0$ と近似できます．

よって運動方程式は近似的に次のようになります．

$$
(M + m) \ddot{x} + m l \ddot{θ} = f \\\\
ml \ddot{x} + ml^2 \ddot{θ} + mgl θ = 0
$$

## 状態方程式を求める

さらにこれを状態空間モデルとして解釈するために，状態ベクトル $X$ を考えます．状態は $x, \dot{x}, θ, \dot{θ}$ の 4 つなので
$$
X = \begin{bmatrix} x \\ \dot{x} \\ θ \\ \dot{θ} \end{bmatrix}
$$
とします．

$X$ の微分を $X$ の線形変換として書き表すために，$\ddot{x}$ と $\ddot{θ}$ を $X$ と $f$ の線型結合で表現する必要があります．これも制御というより式の計算の話なので詳細は省略しますが，実行すると以下のようになります:

$$
\begin{align}
  \ddot{x} &= \frac{mg}{M} θ + \frac{1}{M} f \\\\
  \ddot{θ} &= -\frac{(m + M)g}{M} θ - \frac{1}{M} f
\end{align}
$$

これにより，状態方程式は次のようになります．

$$
\begin{align}
  A &= \begin{bmatrix} 
    0 & 1 & 0 & 0 \\ 
    0 & 0 & \frac{mg}{M} & 0 \\ 
    0 & 0 & 0 & 1 \\ 
    0 & 0 & -\frac{(m + M)g}{M} & 0 
  \end{bmatrix} \\\\
  B &= \begin{bmatrix} 0 \\ \frac{1}{M} \\ 0 \\ -\frac{1}{M} \end{bmatrix} \\\\
  \ddot{X} &= A X + B f
\end{align}
$$

制御出力は $y = (x, θ)^T$ なので，出力方程式は次のようになります．

$$
\begin{align}
  C &= \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \end{bmatrix} \\\\
  D &= \begin{bmatrix} 0 \\ 0 \end{bmatrix} \\\\
  y &= C X + D f
\end{align}
$$


In [5]:
print("Hello World!")

Hello World!